In [2]:
import nbformat as nf
import nbconvert as nc
import os
import subprocess
from glob import glob
from tqdm import tqdm
from datetime import datetime
import shutil as sh

In [17]:
# Finds the ipynb files
files = glob('../live/*/*/*.ipynb')

# A dictionary that runs the function with the value as input
run_keys = {'modified': lambda a: a.format(datetime.today())}

# Where to save the MD posts
save_path = '../../_posts/earth-analytics-python/'

In [18]:
if os.path.exists(save_path):
    sh.rmtree(save_path)

for ifile in tqdm(files):
    # Find file names etc
    path = ifile.split('live/')[-1]
    path_tmp_file = os.path.basename(path)
    file_name = os.path.splitext(path_tmp_file)[0]
    path_folders = os.path.dirname(path)
    
    # Read in notebook, pull meta cell
    ntbk = nf.read(ifile, nf.NO_CONVERT)
    meta = ntbk['cells'][0]
    if meta['cell_type'] == 'raw':

        # Only use lines with key: val pairs
        metalines = meta['source'].split('\n') 
        for ii, ln in enumerate(metalines):
            if '---' in ln:
                continue
            split = ln.split(': ')
            if len(split) < 2:
                continue
            if len(split) > 2:
                ln_type = split[0]
                ln_val = ': '.join(split[1:])
            else:
                ln_type, ln_val = split  
                
            # If we have a function for one of the keys, run it and replace the value
            if ln_type not in run_keys.keys():
                continue
            new_val = eval(run_keys[ln_type](ln_val))
            metalines[ii] = ': '.join([ln_type, new_val])
        
        # Replace the meta cell
        meta['source'] = '\n'.join(metalines)
        ntbk['cells'][0] = meta

    # Write the file
    path_save_tmp_file = os.path.join('./tmp', path_tmp_file)
    nf.write(ntbk, path_save_tmp_file)
    
    # Call nbconvert
    subprocess.call(['jupyter', 'nbconvert', '--to', 'markdown', path_save_tmp_file])
    os.remove(path_save_tmp_file)
    
    # Make new folders if they don't exist
    new_path_folders = os.path.join(save_path, path_folders)
    if not os.path.exists(new_path_folders):
        os.makedirs(new_path_folders)
    
    # Find the folder and the md files that were created, move them to _posts
    for jfile in glob('./tmp/{}*'.format(file_name)):
        sh.move(jfile, new_path_folders)

100%|██████████| 19/19 [01:04<00:00,  3.59s/it]
